# Fáza 3 IAU - Daniela Paľuchová 50%, Mark Bartalos 50% 

Vyriešené veci z predchádzajúcej fázy

In [36]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, precision_score, recall_score

In [5]:
training = pd.read_csv("trenovaci.csv", sep=',')
target = pd.read_csv("trenovaci_target.csv", sep=',')
training

,pct_wild_mouse,screen_height,pct_input,pct_scrandom,pct_scroll_move,pct_click_product_info,wild_mouse_duration,total_load_time,pct_click,screen_width,...,scroll_move_total_rel_distance,page_activity_duration,mouse_move_total_rel_distance,browser_name,pct_rage_click,session_duration,pct_mouse_click,job,race,age
0,59.56562,2,0.05917,48.05874,39.37553,66.21904,13.32802,5.93412,19.29164,2,...,21.28298,18.00126,15.52013,6,41.19641,106.97645,16.06948,3,1,60
1,57.12588,5,0.62610,27.98295,41.97408,58.58313,15.04745,4.19728,12.80114,1,...,18.49826,17.48769,13.88568,4,40.01657,135.12413,18.11505,3,1,60
2,36.23258,5,0.87819,44.02044,46.59896,34.35303,15.35850,2.70523,17.26037,5,...,13.42086,14.73291,16.61463,4,53.41472,92.40541,12.91460,3,1,60
3,58.97383,6,0.81044,30.68743,48.43406,23.69275,15.28388,0.58036,17.37618,2,...,9.02703,16.93843,18.35554,5,33.63685,105.09470,12.93503,3,1,60
4,57.85999,5,2.63628,68.71277,71.42763,60.38453,16.41097,3.62692,17.62559,1,...,15.85784,16.09055,17.97986,5,60.64751,183.27447,13.37094,285,4,97
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8764,37.74552,2,0.63719,61.95955,63.46325,58.79444,15.06358,2.21992,19.95583,3,...,17.53210,16.06603,18.62954,6,40.75720,165.09875,11.59283,3,1,60
8765,56.42685,3,1.59755,28.62159,54.13486,58.15269,15.92552,1.96973,19.25071,2,...,20.48997,14.90900,15.72821,2,47.52506,231.75373,17.37766,360,1,60
8766,44.90033,6,9.91172,33.11470,40.71179,57.39854,17.71756,6.58752,14.44883,2,...,10.07043,16.74973,13.77309,5,43.87094,117.20051,13.22402,3,1,60
8767,54.28177,1,2.28241,43.51204,49.60014,75.44521,16.27053,1.13558,17.38237,2,...,18.51279,16.13312,20.78427,5,67.75252,160.45967,15.12488,3,1,60


## OneR 

In [71]:
def one_r_classifier(X_train, y_train, use_multiple_features=False, columns=None):
    best_feature = None
    best_accuracy = 0
    best_rules = {}

    if columns is None:
        columns = X_train.columns

    for column in columns:
        # Count occurrences of feature-target combinations
        combination_counts = X_train[column].groupby(y_train).value_counts()

        # Create rules: for each feature value, choose the most common target value
        rules = X_train[column].groupby(y_train).apply(lambda x: x.value_counts().idxmax())
        # print(rules)

        # Calculate accuracy
        predictions = X_train[column].map(rules)
        accuracy = (predictions == y_train).mean()

        if accuracy > best_accuracy:
            best_accuracy = accuracy
            best_feature = column
            best_rules = rules

        if use_multiple_features:
            best_rules[column] = rules

    return best_feature, best_rules

# best_feature, best_rules = one_r_classifier(training, target)

def predict(X_test, rules):
    print(list(rules.keys()))
    print(rules.items())
    if len(rules) == 2:  # Single feature
        return X_test[list(rules.keys())[0]].map(rules[list(rules.keys())[0]])
    else:  # Multiple features
        predictions = pd.Series(index=X_test.index, dtype='object')
        
        for column, rule in rules.items():
            predictions = predictions.combine_first(X_test[column].map(rule))
        return predictions


X_train, X_test, y_train, y_test = train_test_split(training, target, test_size=0.2, random_state=42)

# Convert y_train and y_test to DataFrames
y_test = pd.DataFrame(y_test)

attributes = ['pct_scroll_move', 'wild_mouse_duration', 'age']

best_feature, best_rules = one_r_classifier(X_train, y_train['ack'], False)
print(best_rules)
predictions = predict(X_test, best_rules)
print(predictions)


# print("Accuracy:", accuracy_score(y_test, predictions))
# print("Precision:", precision_score(y_test, predictions, average='macro'))
# print("Recall:", recall_score(y_test, predictions, average='macro'))


ack
0    1
1    1
Name: race, dtype: int64
[0, 1]


KeyError: 0

In [99]:
def one_r_classifier(X_train, y_train, use_multiple_features=False):
    best_feature = None
    best_accuracy = 0
    best_rules = {}

    for column in X_train.columns:
        # Create rules: for each feature value, choose the most common target value
        rules = X_train[column].groupby(y_train).apply(lambda x: x.value_counts().idxmax())

        # Calculate accuracy
        predictions = X_train[column].map(rules)
        accuracy = (predictions == y_train).mean()

        if not use_multiple_features and accuracy > best_accuracy:
            best_accuracy = accuracy
            best_feature = column
            best_rules = {column: rules}

        if use_multiple_features:
            best_rules[column] = rules

            # if not use_multiple_features:
            #     return best_feature, best_rules
            # else:
            #     return None, best_rules

    return best_feature if not use_multiple_features else None, best_rules

def predict(X_test, rules):
    if len(rules) == 1:  # Single feature
        feature = next(iter(rules))
        return X_test[feature].map(rules[feature]).fillna(-1)
    else:  # Multiple features
        predictions = pd.Series(index=X_test.index, dtype='object')
        for column, rule in rules.items():
            column_predictions = X_test[column].map(rule)
            predictions = predictions.combine_first(column_predictions)
        return predictions.fillna(-1)

    # Example Data Preparation
X_train, X_test, y_train, y_test = train_test_split(training, target, test_size=0.2, random_state=42)

    # Train 1R classifier for single feature
best_feature, rules = one_r_classifier(X_train, y_train['ack'])
predictions = predict(X_test, rules)

predictions

    # Train 1R classifier for multiple features
_, rules_multiple = one_r_classifier(X_train, y_train['ack'], use_multiple_features=True)
predictions_multiple = predict(X_test, rules_multiple)
print(predictions_multiple)


# Evaluate classifier
print("Accuracy:", accuracy_score(y_test, predictions_multiple))
print("Precision:", precision_score(y_test, predictions_multiple, average='macro'))
print("Recall:", recall_score(y_test, predictions_multiple, average='macro'))


7011   -1.0
2317    3.0
2602    3.0
3161    2.0
1017    2.0
       ... 
5097    2.0
3685    1.0
2499    1.0
5388    1.0
6350    3.0
Length: 1754, dtype: float64
Accuracy: 0.29817559863169896
Precision: 0.08958547447756081
Recall: 0.09322638146167557


/var/folders/g0/hx6lq77547x54x2r4mb51vx80000gn/T/ipykernel_5915/2012252868.py:37: FutureWarning: The behavior of array concatenation with empty entries is deprecated. In a future version, this will no longer exclude empty items when determining the result dtype. To retain the old behavior, exclude the empty entries before the concat operation.
  predictions = predictions.combine_first(column_predictions)
/Users/markbartalos/.virtualenvs/iau/lib/python3.11/site-packages/sklearn/metrics/_classification.py:1469: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/Users/markbartalos/.virtualenvs/iau/lib/python3.11/site-packages/sklearn/metrics/_classification.py:1469: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _wa